# Utils

In [ ]:
# | default_exp utils

In [ ]:
# | export

from vid_chains.imports import *
import math
import operator

/home/rafay/miniconda3/envs/jbt_proj/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
#| hide

%load_ext autoreload
%autoreload 2
%reload_ext autoreload

In [ ]:
# | export

def setify(o): return o if isinstance(o,set) else set(list(o))

def _get_files(p, fs, extensions=None):
    p = Path(p)
    res = [p/f for f in fs if not f.startswith('.')
           and ((not extensions) or f'.{f.split(".")[-1].lower()}' in extensions)]
    return res

def get_files(path, extensions=None, recurse=True, folders=None, followlinks=True):
    "Get all the files in `path` with optional `extensions`, optionally with `recurse`, only in `folders`, if specified."
    if folders is None:
        folders = list([])
    path = Path(path)
    if extensions is not None:
        extensions = setify(extensions)
        extensions = {e.lower() for e in extensions}
    if recurse:
        res = []
        for i,(p,d,f) in enumerate(os.walk(path, followlinks=followlinks)): # returns (dirpath, dirnames, filenames)
            if len(folders) !=0 and i==0: d[:] = [o for o in d if o in folders]
            else:                         d[:] = [o for o in d if not o.startswith('.')]
            if len(folders) !=0 and i==0 and '.' not in folders: continue
            res += _get_files(p, f, extensions)
    else:
        f = [o.name for o in os.scandir(path) if o.is_file()]
        res = _get_files(path, f, extensions)
    return list(res)

def list_map(l, m):
    return list(pd.Series(l).apply(m))

def get_image_files(path, recurse=True, folders=None, map_fn=None,
                    sort_key=None, reverse=False, shuffle=False):
    "Get image files in `path` recursively, only in `folders`, if specified."
    l = get_files(path, extensions=image_extensions, recurse=recurse, folders=folders)
    if sort_key is not None:
        l = sorted(l, key=sort_key)
    if reverse:
        l = list(l)[::-1]
    if shuffle:
        random.shuffle(l)
    if map_fn is not None:
        return list_map(l, map_fn)
    return l


def load_obj_model(name="yolov8n.pt"):
    return YOLO(name)


def detect_objects(model, img):
    res = model(img, stream=True)
    return [{"boxes": r.boxes.data.detach().cpu().tolist()} for r in res]

def get_width (l):
    w = l[2]-l[0]
    return w

def list_widths (obj):
    w=[]
    for i in range(0,len(obj.get('boxes'))):
      l=[]
      for j in range(0,6):
        l.append(obj.get('boxes')[i][j])
      if l[5] == 39.0: #very specific test case for bottles so will ignore other objects, will remove this in the future
        width = get_width(l)
        w.append(width)
    return w

def centroid (l):
    t = []
    cx = (l[0]+l[2])/2.0
    cy = (l[1]+l[3])/2.0
    t.append(cx)
    t.append(cy)
    return t


def list_centroids (obj):
    c=[]
    for i in range(0,len(obj)):
      l=[]
      for j in range(0,4):
        l.append(obj[i][j])
      centre = centroid(l)
      c.append(centre)
    return c

def inter_dist(obj):
   
    c = list_centroids(obj)
    dis = []
    st = []
    for i in range(0,len(c)):
        for j in range(i+1,len(c)):
            #st.append("Distance b/w object "+str(i)+" and object "+str(j))
            #st.append("D("+str(i)+","+str(j)+")")
            dis.append(math.dist(c[i],c[j]))
    #return st,dis
    return dis

def focal_len_to_px (focal_len, sensor_px):
    return round((focal_len/sensor_px)*1000)

def camera_to_obj_dist (focal_length_px, obj, real_width):

    widths = list_widths(obj)
    dists = []
    for w in widths:
        distance = (real_width * focal_length_px) / w
        dists.append(distance)

    return dists

### Helper functions for distances between masks by getting extreme points, 
# all used in calculate_distance_masks i.e the main function 

def get_coords(mask_folder):
    mask_fnames = get_image_files(mask_folder)
    list_coords = []
    # generates a list (y,x) coords..
    for mask_fname in mask_fnames:
        mask = cv2.imread(str(mask_fname))
        mask = cv2.cvtColor(mask, cv2.COLOR_RGB2GRAY)
        x = np.where(mask == 255)
        coords = np.asarray(x).T.tolist()
        for c in coords:
            c[0], c[1] = c[1], c[0]
        list_coords.append(coords)
    return list_coords

def sort_cords (coords):
    return sorted(coords, key=operator.itemgetter(0))

def get_edges(coords):
    points = []
    for c in coords:
        x_min = min([sublist[0] for sublist in c])
        y_min = min([sublist[1] for sublist in c])
        x_max = max([sublist[0] for sublist in c])
        y_max = max([sublist[1] for sublist in c])
        temp = x_min, y_min, x_max, y_max
        points.append(list(temp))
    return points

def min_dict (eds, coords):
    ob_dict = {}
    for i in range(0,len(eds)):
        if i == 0:
            for c in coords:
                for cr in c:
                #if i == 0:
                    if cr[0] == eds[i][2]:
                        #t.append(cr)
                        ob_dict[str(i)] = [list(cr)]
                        break
        else:
            for c in coords:
                for cr in c:
                    if cr[0] == eds[i][0]:
                        #t.append(cr)
                        ob_dict[str(i)] = [list(cr)]
                        break
    return ob_dict

def max_dict (eds, coords):
    ob_max = {}
    for i in range(1, len(eds)-1):
        for c in coords:
            for cr in c:
                if cr[0] == eds[i][2]:
                    #t.append(cr)
                    ob_max[str(i)] = [list(cr)]
                    break
    return ob_max

def merge_dicts (ob_dict, ob_max):
    for i in range(1,len(ob_dict)-1):
        t = ob_dict.get(str(i)),ob_max.get(str(i))
        ob_dict[str(i)] = flatten_list(t)
    return ob_dict

def calculate_distance_masks (mask_folder):
    coords = get_coords("../mask_imgs/")
    coords = sort_cords(coords)
    edges = get_edges(coords)
    ob_dict = min_dict(edges, coords)
    ob_max = max_dict(edges, coords)
    ob_dict = merge_dicts(ob_dict, ob_max)

    dists = []
    for i in range(0,len(ob_dict)):
        for j in range(i+1,len(ob_dict)):
            #st.append("Distance b/w object "+str(i)+" and object "+str(j))
            #st.append("D("+str(i)+","+str(j)+")")
            if i == 0:
                dists.append(math.dist(ob_dict[str(i)][0],ob_dict[str(j)][0]))
                #dists.append(math.dist(c[i],c[j]))
            else:
                dists.append(math.dist(ob_dict[str(i)][1],ob_dict[str(j)][0]))
    return dists

###################################################################################

def yolo_objects_format(objects):
    objects = [obj.get("boxes") for obj in objects]
    objects = flatten_list(objects)
    return objects

def flatten_list(l):
    try:
        return sum(l, [])
    except:
        return sum(l, ())


In [ ]:
# | hide
# | eval: false

img_path = "../exp_imgs/"

model = load_obj_model()
objects = detect_objects(model, img_path)


image 1/1 /home/rafay/vid_chains/nbs/../exp_imgs/bottle.jpg: 480x640 1 bench, 2 bottles, 604.1ms
Speed: 14.9ms preprocess, 604.1ms inference, 20.3ms postprocess per image at shape (1, 3, 480, 640)


In [ ]:
# | hide
# | eval: false


objects

[{'boxes': [[196.56224060058594,
    1210.6712646484375,
    475.10223388671875,
    1623.388916015625,
    0.5954297780990601,
    39.0],
   [468.8839111328125,
    0.817108154296875,
    2632.79833984375,
    926.1826782226562,
    0.46843209862709045,
    13.0],
   [3620.0390625,
    1252.4671630859375,
    3909.64697265625,
    1658.0111083984375,
    0.3052195906639099,
    39.0]]}]

In [ ]:
# | hide
# | eval: false
focal_length_px = focal_len_to_px(6.04, 0.7) #redmi note 10 pro camera specifications
real_width = 0.0635 #of bottle
cam_dists = [camera_to_obj_dist(focal_length_px, objs, real_width) for objs in objects]
cam_dists

[[1.967191474141819, 1.8920115120625443]]

In [ ]:
# | hide
# | eval: false
objs = yolo_objects_format(objects)
dists = inter_dist(objs)
dists

[1544.4955277947797, 3429.2236530738533, 2425.974258601772]

In [ ]:
# | hide
# | eval: false

# Distances between centroids of masks

coords = get_coords("../mask_imgs/")
edges = get_edges(coords)
dists_centroids = inter_dist(edges)
dists_centroids

[441.99915158289616, 796.0001570351604, 355.15137617641295]

In [ ]:
# | hide
# | eval: false

# distances between extreme points of masks 
distances = calculate_distance_masks("../mask_imgs/")
distances

[336.31532822635364, 667.3290043149631, 181.0]

In [ ]:
# | hide
# | eval: false

#import matplotlib.pyplot as plt
#image = plt.imread("../mask_imgs/sample_1_mask0.png")
#plt.imshow(image, interpolation="none")
#plt.show()

In [ ]:
# | hide
import nbdev

nbdev.nbdev_export()